<img src="https://raw.githubusercontent.com/NCAR/dask-tutorial/main/images/NCAR-contemp-logo-blue.png"
     width="750px"
     alt="NCAR logo"
     style="vertical-align:middle;margin:30px 0px"/>

# Dask Chunking - Best Practices

**ESDS Dask tutorial | 06 February, 2023**  

Negin Sobhani, Brian Vanderwende, Deepak Cherian, Ben Kirk  
Computational & Information Systems Lab (CISL)  
[negins@ucar.edu](mailto:negins@ucar.edu), [vanderwb@ucar.edu](mailto:vanderwb@ucar.edu)

------

### In this tutorial, you will learn:

* Basic rules of thumb for chunking
* The importance of conforming to file chunks
* The impact of rechunking in the computational pipeline

**Related Documentation**

* [Dask Chunking Documentation](https://docs.dask.org/en/stable/array-chunks.html)
* [Choosing Chunk Sizes Blog Post](https://blog.dask.org/2021/11/02/choosing-dask-chunk-sizes)
* [Xarray Chunking Documentation](https://docs.xarray.dev/en/stable/user-guide/dask.html#chunking-and-performance)

---

## Chunking Considerations

Determining the best approach for sizing your Dask chunks can be tricky and often requires intuition about both Dask and your particular dataset. There are various considerations you may need to account for depending on your workflow:

* The size (in bytes) of your chunks vs your number of workers
* The chunk layout of data read from disk (formats like HDF5, Zarr)
* The access patterns of your computational pipeline

**Dask Array with NumPy array chunks...**

<img src="https://docs.dask.org/en/stable/_images/dask-array.svg" width=500px alt="Dask Array Chunks">


----

### Starting up our PBS Cluster

To demonstrate the affects of different chunking strategies, let's instantiate a `PBSCluster` with 4 workers

In [1]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

In [2]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk23-chunking',
    cores = 1,
    memory = '10GiB',
    processes = 1,
    local_directory = '/glade/scratch/vanderwb/temp/dask/spill/pbs.$PBS_JOBID',
    resource_spec = 'select=1:ncpus=1:mem=10GB',
    queue = 'casper',
    walltime = '30:00',
    interface = 'ib0'
)

/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42815 instead
  warnings.warn(


In [3]:
# Sanity-check our setup
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-wk23-chunking
#PBS -q casper
#PBS -A SCSG0001
#PBS -l select=1:ncpus=1:mem=10GB
#PBS -l walltime=30:00
#PBS -e dask-worker-logs/
#PBS -o dask-worker-logs/
#PBS -j oe

/glade/u/apps/opt/conda/envs/npl-2023a/bin/python -m distributed.cli.dask_worker tcp://10.12.206.46:41979 --nthreads 1 --memory-limit 10.00GiB --name dummy-name --nanny --death-timeout 60 --local-directory /glade/scratch/vanderwb/temp/dask/spill/pbs.$PBS_JOBID --interface ib0



In [4]:
client = Client(cluster)

In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/vanderwb/Casper/proxy/42815/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/vanderwb/Casper/proxy/42815/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.46:41979,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/vanderwb/Casper/proxy/42815/status,Total threads: 0
Started: Just now,Total memory: 0 B


----

## Chunk size - Load balancing vs. Overhead

There is always an optimal chunk size given your hardware setup and computation problem that is neither too big nor too small. Finding this chunk size often requires some trial and error, but it is helpful to know what you are looking to avoid:

* **Too small** - if your chunks are too small, you will end up spending a significant and wasteful amount of time waiting for Dask to perform overhead (scheduling tasks, data communication) relative to the time spent computing
* **Too large** - you run the risk of spilling to dask or memory failures and the scheduler also has a more difficult time load balancing

The following rules of thumb are known, but it will vary according to your workflow:

|Too Small|Possibly Too Small|Optimal|Too Large|
|-|-|-|-|
|< 1 MB|1-100 MB|100 MB - 1 GB|> Spill threshold|

In practice, using chunks close to 0.1-0.5 GB in size works well.

#### Let's test these rules of thumb...

In [6]:
# Spin up workers on our PBS cluster
cluster.scale(4)
client.wait_for_workers(4)

For this exercise, we will simply generate a random number **Dask Array** of sufficient size that it would not fit in our login session memory. Let's try different chunking strategies.

In [7]:
import dask.array as da

In [8]:
t = da.random.random((60000, 72000), chunks = (30000,36000))
t

dask.array<random_sample, shape=(60000, 72000), dtype=float64, chunksize=(30000, 36000), chunktype=numpy.ndarray>

These chunks are too large. They will exceed our spill threshold (0.6-0.7) and even briefly exceed our pause limit (0.8). The only thing working in our favor in this configuration is that non-aggregation tasks should be well-balanced among the 4 workers with 4 chunks, and we have a short task graph.

In [9]:
task = t.mean()
task.dask

HighLevelGraph with 3 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x2b82660b3c70>
 0. random_sample-99d384ebf7643e046beb2659e21d477c
 1. mean_chunk-1db7ddb1bae521842cca9af0d431c4c5
 2. mean_agg-aggregate-b00133a3cd939d8bebad9aae1408daf7

In [10]:
%%time
result = task.compute()

CPU times: user 608 ms, sys: 143 ms, total: 751 ms
Wall time: 10.4 s


In this next configuration, we end up specifying a configuration with very small chunks relative to the problem size. We will not come close to the memory limits, but we will incur significant overhead relative to our computational task.

In [11]:
t = da.random.random((60000, 72000), chunks = (1000,1000))
t

dask.array<random_sample, shape=(60000, 72000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [12]:
task = t.mean()
task.dask

HighLevelGraph with 9 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x2b826639b580>
 0. random_sample-d864cc98d7b0c810e3981396d0a31505
 1. mean_chunk-2d6be3ffa6620a03b940271bbcfde966
 2. mean_combine-partial-29b2186694d79f2d7731b08940ea330c
 3. mean_combine-partial-58ad99c24bbf255733617407d84da342
 4. mean_combine-partial-455a2b59687d27f757814666f78b9772
 5. mean_combine-partial-6cd2c59ea14367c391f8098f19de908a
 6. mean_combine-partial-ae112b9d4ead8774877687e1fe1a34f6
 7. mean_combine-partial-0e2db5ea9697d821664c1bb0b1bc891f
 8. mean_agg-aggregate-2ed8945b01b8b27bf89378b4d820bdf5

In [13]:
%%time
result = task.compute()

CPU times: user 5.78 s, sys: 262 ms, total: 6.04 s
Wall time: 12.8 s


Next, we will choose chunk sizes that fall in our expected "optimal" range of `100 MiB - 1 GiB`. We should be allowing Dask to distribute work efficiently but not imposing a high overhead...

In [14]:
t = da.random.random((60000, 72000), chunks = (10000,6000))
t

dask.array<random_sample, shape=(60000, 72000), dtype=float64, chunksize=(10000, 6000), chunktype=numpy.ndarray>

In [15]:
%%time
result = t.mean().compute()

CPU times: user 796 ms, sys: 35.3 ms, total: 831 ms
Wall time: 9.39 s


----

## Matching chunking in a netCDF4 file

If you are using a chunked data format, it is best to specify Dask chunks which equal to or (better-yet) multiples of the chunk shape on disk. If chunk sizes aren't multiples of disk chunks, you risk unnecessary additional reads of data as multiple disk chunks will need to be read to populate each Dask chunk. This can be very inefficient!

#### Inspecting file chunking

The exact process for checking file chunking depends on the format. Using the netCDF4 Python module, we can query the chunking parameters of any variable in a netCDF4 file.

*Classic netCDF files do not support chunking!*

In [16]:
import netCDF4 as nc

We will use a data file from a model forecast dataset over the Arctic:

In [17]:
my_file = '/glade/collections/rda/data/ds631.1/asr15.fcst3.3D/asr15km.fct.3D.20120916.nc'

Once we open the *dataset* (nc4 file), we can reference a variable of interest using a dictionary key and then get the dimensions of that variable using `get_dims()`.

In [18]:
nc_data = nc.Dataset(my_file)
nc_data['CLDFRA'].get_dims()

(<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'Time', size = 8,
 <class 'netCDF4._netCDF4.Dimension'>: name = 'num_metgrid_levels', size = 34,
 <class 'netCDF4._netCDF4.Dimension'>: name = 'south_north', size = 720,
 <class 'netCDF4._netCDF4.Dimension'>: name = 'west_east', size = 720)

We can then use the `chunking()` method to get our chunk size for each dimension:

In [19]:
nc_data['CLDFRA'].chunking()

[1, 16, 355, 355]

### Specifying chunks using Xarray

Now that we understand our file chunks, we can specify a preferred chunk size to `open_dataset`. Note that if we use the `chunks` parameter, any dimension we don't mention will be spanned in its entirety for chunks.

In [20]:
import xarray as xr

In [29]:
# Open dataset using chunking along Time dimension
ds = xr.open_dataset(my_file, chunks = {'Time' : 1})

Since we are only specifying a chunk size for Time, this should be equivalent to the following chunk shape:
```python
chunks = {'Time' : 1,
          'num_metgrid_levels' : -1,
          'south_north' : -1,
          'west_east' : -1 }
```
We can confirm that our chunks look as intended using the DataArray *repr*:

In [22]:
ds.CLDFRA

<xarray.DataArray 'CLDFRA' (Time: 8, num_metgrid_levels: 34, south_north: 720,
                            west_east: 720)>
dask.array<open_dataset-2d5c51faf2dc21d9877d4227a4fb805dCLDFRA, shape=(8, 34, 720, 720), dtype=float32, chunksize=(1, 34, 720, 720), chunktype=numpy.ndarray>
Coordinates:
  * Time     (Time) datetime64[ns] 2012-09-16 ... 2012-09-16T21:00:00
    XLAT     (south_north, west_east) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    XLONG    (south_north, west_east) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
Dimensions without coordinates: num_metgrid_levels, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    units:        1.0
    stagger:      
    long_name:    CLOUD FRACTION

**Note:** You can also retrieve the file chunk size from Xarray itself, but it is not shown in the above repr. Use the following DataArray (variable) attribute instead:

In [23]:
ds.CLDFRA.encoding["chunksizes"]

(1, 16, 355, 355)

Now let's benchmark various chunk configurations. Our initial guess achieves the recommended ratio of >= 2 chunks per worker, but does use multiples of the file chunk size except in the time dimension.

For this benchmark, we will find the maximum cloud fraction across vertical levels at all locations and times.

In [30]:
%%time
result = ds.CLDFRA.max(dim = "num_metgrid_levels").compute()

CPU times: user 163 ms, sys: 28.7 ms, total: 192 ms
Wall time: 1.73 s


Notice above that this file has chunking that does not divide evenly into the dimension sizes. We can specify that our chunks match the file chunks directly, but this will leave "remainder" chunks and will slightly increase overhead.

In [31]:
ds = xr.open_dataset(my_file, chunks = {'Time' : 1, "num_metgrid_levels" : 16,
                                        "south_north" : 355, "east_west" : 355})
ds.CLDFRA

<xarray.DataArray 'CLDFRA' (Time: 8, num_metgrid_levels: 34, south_north: 720,
                            west_east: 720)>
dask.array<open_dataset-a2a8daca35d216b566b6c638540eeed3CLDFRA, shape=(8, 34, 720, 720), dtype=float32, chunksize=(1, 16, 355, 720), chunktype=numpy.ndarray>
Coordinates:
  * Time     (Time) datetime64[ns] 2012-09-16 ... 2012-09-16T21:00:00
    XLAT     (south_north, west_east) float32 dask.array<chunksize=(355, 720), meta=np.ndarray>
    XLONG    (south_north, west_east) float32 dask.array<chunksize=(355, 720), meta=np.ndarray>
Dimensions without coordinates: num_metgrid_levels, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    units:        1.0
    stagger:      
    long_name:    CLOUD FRACTION

In [32]:
%%time
result = ds.CLDFRA.max(dim = "num_metgrid_levels").compute()

CPU times: user 171 ms, sys: 34.8 ms, total: 206 ms
Wall time: 1.25 s


The most problematic case occurs when we have chunk sizes that are smaller than the file chunks in one or more dimensions. Let's evaluate the impact by using progressively smaller vertical level ranks:

In [33]:
# Using half the file chunk size in the vertical (same number of chunks)
ds = xr.open_dataset(my_file, chunks = {"Time" : 4, "num_metgrid_levels" : 8})

In [34]:
%%time
result = ds.CLDFRA.max(dim = "num_metgrid_levels").compute()

CPU times: user 202 ms, sys: 30.5 ms, total: 233 ms
Wall time: 2.44 s


In [35]:
# Use 1/4 the chunk size in the vertical
ds = xr.open_dataset(my_file, chunks = {"Time" : 8, "num_metgrid_levels" : 4})

In [36]:
%%time
result = ds.CLDFRA.max(dim = "num_metgrid_levels").compute()

CPU times: user 290 ms, sys: 40.4 ms, total: 330 ms
Wall time: 4.09 s


It is also possible to use "auto" chunking, whereby the DataArray chunks are calculated for you. Are these optimal?

In [37]:
# Open dataset using auto-chunking
ds = xr.open_dataset(my_file, chunks = 'auto')

In [38]:
ds.CLDFRA

<xarray.DataArray 'CLDFRA' (Time: 8, num_metgrid_levels: 34, south_north: 720,
                            west_east: 720)>
dask.array<open_dataset-2475367054d27aaab126f113eb08b50aCLDFRA, shape=(8, 34, 720, 720), dtype=float32, chunksize=(5, 23, 539, 539), chunktype=numpy.ndarray>
Coordinates:
  * Time     (Time) datetime64[ns] 2012-09-16 ... 2012-09-16T21:00:00
    XLAT     (south_north, west_east) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
    XLONG    (south_north, west_east) float32 dask.array<chunksize=(720, 720), meta=np.ndarray>
Dimensions without coordinates: num_metgrid_levels, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    units:        1.0
    stagger:      
    long_name:    CLOUD FRACTION

In [39]:
%%time
result = ds.CLDFRA.max(dim = "num_metgrid_levels").compute()

CPU times: user 239 ms, sys: 47.5 ms, total: 287 ms
Wall time: 2.94 s


**No! Avoid using auto chunking for files written in chunks!**

## Rechunking is expensive

There are various reasons Dask might need to rechunk data, but in any case, it can be an expensive operation with a large amount of communication required between workers.

**Scenario:** We wish to get the mean difference between two versions of a model for the same case study. Unfortunately, while the grids match for each version, the file chunk size used was different.

Here, we will emulate the scenario with Dask Arrays...

In [40]:
old_run = da.random.random((800,600,60,20), chunks = (400,300,30,1))

In [41]:
old_run

,Array,Chunk
Bytes,4.29 GiB,27.47 MiB
Shape,"(800, 600, 60, 20)","(400, 300, 30, 1)"
Count,1 Graph Layer,160 Chunks
Type,float64,numpy.ndarray


In [42]:
new_run = da.random.random((800,600,60,20), chunks = (800,600,10,1))

In [43]:
new_run

,Array,Chunk
Bytes,4.29 GiB,36.62 MiB
Shape,"(800, 600, 60, 20)","(800, 600, 10, 1)"
Count,1 Graph Layer,120 Chunks
Type,float64,numpy.ndarray


Let's set up and analyse (via a high-level task graph), the operations we will need to do to retrieve a mean-squared difference/error between our two datasets.

In [44]:
# Calculate the mean squared difference
mse_graph = ((old_run - new_run) ** 2).sum() / old_run.size

In [45]:
mse_graph.dask

layer_type,MaterializedLayer
is_materialized,True
number of outputs,120
shape,"(800, 600, 60, 20)"
dtype,float64
chunksize,"(800, 600, 10, 1)"
type,dask.array.core.Array
chunk_type,numpy.ndarray
layer_type,MaterializedLayer
is_materialized,True
number of outputs,960


Note the two rechunking operations near the beginning of our task graph. Because our data arrays are chunked differently, Dask must rechunk first to avoid slowing down operations with large data transfers between workers. It is good that Dask does this, but rechunking is still expensive...

In [46]:
%%time
mse_graph.compute()

CPU times: user 2.09 s, sys: 81.1 ms, total: 2.17 s
Wall time: 6.03 s


0.16667671401503523

In most circumstances, we will want to rechunk this data ourselves manually, and then save state (probably by creating a new rechunked data file). This one-time cost means we will not need to rechunk again in the future.

In our scenario, we would likely rechunk the old run data, since we expect all future runs will have the new chunking.

```python
old_run_rechunked = old_run.rechunk((800,600,10,1))
```

Once this is done in a conversion workflow, we could load the rechunked data in our current workflow.

In [47]:
old_run = da.random.random((800,600,60,20), chunks = (800,600,10,1))

In [48]:
# Calculate the mean squared difference
mse_graph = ((old_run - new_run) ** 2).sum() / old_run.size

In [49]:
mse_graph.dask

layer_type,MaterializedLayer
is_materialized,True
number of outputs,120
shape,"(800, 600, 60, 20)"
dtype,float64
chunksize,"(800, 600, 10, 1)"
type,dask.array.core.Array
chunk_type,numpy.ndarray
layer_type,MaterializedLayer
is_materialized,True
number of outputs,120


In [50]:
%%time
mse_graph.compute()

CPU times: user 446 ms, sys: 21.8 ms, total: 468 ms
Wall time: 3.12 s


0.1666746512370409

In [51]:
client.shutdown()

## Takeaway Message

Chunking is fundamental to Dask and its blocked-algorithm approach, so don't ignore intelligently sizing your data chunks. Finding the perfect chunk size is not the goal, but neglecting simple rules of thumb can lead to massive performance penalties when aggregated over a complex multipart analysis.